### Wstęp do xgboost

Zaczynamy od importów

In [ ]:
! pip install xgboost     # czysty Python
! conda install xgboost   # anaconda

In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import time

Zaczniemy od przejrzenia zestawu danych. Jest to problem regresyjny - jest to cennik mieszkań.

URL: https://www.kaggle.com/shivachandel/kc-house-data

Dane znajdują się w pliku `kc_house_data.csv`.

In [ ]:
data = pd.read_csv("./data/kc_house_data.csv")
data.head()

In [ ]:
# date - split to year
data.info()

XGBoost jest oddzielną biblioteką niż sklearn. Jest ona kompatybilna z sklearn, natomiast posiada kilka rozszerzeń specyficznych do tej biblioteki. Rzućmy okiem na obiekt reprezentujący model, który będziemy uczyć:

In [ ]:
xgb.XGBRegressor?

Część parametrów jest podobna do lasu losowego (`n_estimators`), część odzwierciedla równania, na których bazowana jest funkcja straty. Najbardziej interesujące parametry:

- n_estimators
- learning_rate
- booster
- objective

Więcej na: https://xgboost.readthedocs.io/en/latest/parameter.html

### Przykładowy trening

In [ ]:
y = data['price']
x = data.drop(['id', 'price', 'date', 'zipcode'], axis=1)

Podział danych na treningowe oraz testowe

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
x_train.info()

In [ ]:
x_train.bedrooms.value_counts()

In [ ]:
x_train.waterfront.value_counts()

In [ ]:
x_train.view.value_counts()

Stworzenie regresora w XGBoost. Istotne parametry: `objective="reg:squarederror"`, `booster="gbtree"`. Seed to ustalenie ziarna losowania (reprodukowalność rozwiązań)

In [ ]:
xg_reg = xgb.XGBRegressor(random_state=123, n_estimators=5, max_depth=7)

In [ ]:
xg_reg

Trening modelu.

In [ ]:
%%time

start = time.time()

xg_reg.fit(x_train, y_train)

end = time.time()
print("time: ", end - start)

Predykcja danych testowych

In [ ]:
y_pred = xg_reg.predict(x_test)

In [ ]:
y_pred

In [ ]:
xg_reg.score(x_test, y_test) #R^2

Obliczmy błąd RMSE (root mean squared error)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sklearn.metrics

In [ ]:
# np.sqrt(np.mean(((y_test - y_pred) ** 2)))

# mean_squared_error(GROUNDTRUTH, PREDYKCJE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = sklearn.metrics.mean_absolute_error(y_test, y_pred)

In [ ]:
print("RMSE - gbtree: ", rmse)
print("MAE - gbtree: ", mae)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(np.abs(y_test - y_pred), bins=50);

In [ ]:
y_pred = np.array([200, 240])
y_true = np.array([150, 140])

In [ ]:
mae = np.mean(np.abs(y_true - y_pred))
mse = np.mean((y_true - y_pred) ** 2)
rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

In [ ]:
print(mae, mse, rmse)

In [ ]:
from matplotlib.pylab import rcParams
##set up the parameters
rcParams['figure.figsize'] = 10, 8

xgb.plot_importance(xg_reg)

In [ ]:
xgb.plot_importance?

### Alternatywne boostery

Istnieje więcej niż jedna metoda boostowania. `gbtree` opiera się o drzewa, możemy użyć modelu liniowego zamiast drzewa.
Stworzenie regresora (`booster="gblinear"`).

In [ ]:
xg_reg_lin = xgb.XGBRegressor(objective="reg:squarederror", seed=123, booster="gblinear", n_estimators=50)

In [ ]:
xg_reg_lin.fit(x_train, y_train)

In [ ]:
y_pred = xg_reg_lin.predict(x_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
print("RMSE: ", rmse)

### Nieliniowe przekształcenie

Jeżeli wykreślimy cenę, zauważymy, że dystrybucja cen nie przypomina rozkładu normalnego. Jest to bardzo niezrównoważony rozkład.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
##set up the parameters
rcParams['figure.figsize'] = 10, 7

plt.hist(data['price'], bins=20)
_ = plt.title("Histogram of prices")

In [ ]:
data['price'].head()

In [ ]:
np.log10(data['price'].head())

In [ ]:
min(np.log10(data['price'].head())), max(np.log10(data['price'].head()))

In [ ]:
10 ** np.log10(data['price'].head())

Spróbujmy przekształcić ten zestaw danych - np. przy pomocy przekształcenia logarytmicznego (dla danych finansowych często jest to pożyteczne przekształcenie).

In [ ]:
plt.hist(np.log10(data['price']), bins=20)
_ = plt.title("Histogram of log-prices")

In [ ]:
# wykreślenie cen z osią X odpowiadającą rzeczywistej cenie, a nie wykładnikowi
# jeden sposób: https://stackoverflow.com/questions/6855710/how-to-have-logarithmic-bins-in-a-python-histogram
# drugi sposób: https://kite.com/python/examples/1870/matplotlib-change-x-axis-tick-labels

# wykorzystano przekształcenie odwrotne, które jest funkcją wykładniczą np.exp

plt.hist(np.log10(data['price']), bins=20)
_ = plt.title("Histogram of log-prices - log=True")
plt.xlabel("House price")
plt.ylabel("Number of houses")
_ = plt.xticks(np.arange(5, 7), np.floor(10 ** (np.arange(5, 7))))

# Zadanie 1

Wykorzystując przekształcenie logarytmiczne sprawdź w jaki sposób poprawia to modele. Sprawdź `gbtree` oraz `gblinear` (analogiczie do modeli powyżej).

Aby Twój model mógł być porównany z poprzednim, musisz wykorzystać tą samą miarę błędu. W tym celu - dokonaj przekształcenia odwrotnego na wynikach (funkcji wykładniczej) a następnie porównaj wyniki regresji w oryginalnej skali oraz wartości docelowych dla tych przykładów.

In [ ]:
# x, y

y_transformed = np.log10(y)
y_transformed

In [ ]:
train_x, test_x, train_y, test_y, train_yt, test_yt = train_test_split(
    x, y, y_transformed, test_size=0.2, random_state=123
)
test_y.shape, test_yt.shape